In [1]:
import requests as rq
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import os


In [2]:
features = ["発行日", "巻", "号", "タイトル", "著者1", "著者2", "著者3",
            "doi", "キーワード1", "キーワード2", "キーワード3", "キーワード4", "キーワード5",  "キーワード6","抄録",
            "引用文献1","引用文献2", "引用文献3","引用文献4","引用文献5","引用文献6","引用文献7","引用文献8","引用文献9","引用文献10","引用文献11","引用文献12","引用文献13","引用文献14","引用文献15","引用文献16","引用文献17","引用文献18","引用文献19","引用文献20","引用文献21","引用文献22","引用文献23","引用文献24","引用文献25","引用文献26","引用文献27","引用文献28","引用文献29","引用文献30","引用文献31","引用文献32","引用文献33","引用文献34","引用文献35","引用文献36","引用文献37","引用文献38","引用文献39","引用文献40","引用文献41","引用文献42","引用文献43","引用文献44","引用文献45","引用文献46","引用文献47","引用文献48","引用文献49","引用文献50"
            ] #アウトプット用のカラムの指定
output_all = pd.DataFrame(columns = features)
ind_num = 1 #出力用のインデックス No.

kan_start = 1
kan_end = 40

kans = list(range(kan_start,kan_end+1,1)) #第何巻かの指定

for kan in kans:
    # 号を指定するところ。　
    gous = list(range(1,5,1))  #star, stop, 刻み。stopが4だと、3までが対象

    # 雑誌のURLを指定するところ。対象誌毎に変更
    soup_URL = "https://www.jstage.jst.go.jp/article/arp/{0}/{1}/.*/_article/-char/ja" 

    for gou in gous:
        # 雑誌のURLを指定するところ。対象誌毎に変更
        url = "https://www.jstage.jst.go.jp/browse/arp/{0}/{1}/_contents/-char/ja".format(kan, gou)
        print("vol",kan,"issue",gou)
        
        r = rq.get(url) #情報の取得
        time.sleep(1)
        soup = BeautifulSoup(r.text, "html.parser") #Beautifulsoupでの情報解析
        elems = soup.find_all(href=re.compile(soup_URL.format(kan, gou))) #リンクの中から関連するもののみを取得
        #必要なもののみ選択するため、for文で回す
        for elem in elems:
            elem_str = str(elem) #検索のために文字列を準備

            #"表紙"or"目次"or"OTHEROS"or"裏表紙"など規定の文字列が含まれていたらスキップ( or "プログラム" in elem_str)
            if "表紙" in elem_str or "目次" in elem_str or "[OTHERS]" in elem_str or "裏表紙" in elem_str \
            or "CONTENTS" in elem_str or "会誌" in elem_str or "規程" in elem_str or "議事録" in elem_str \
            or "学会だより" in elem_str or "奥付" in elem_str or "賛助会員" in elem_str or "お知らせ" in elem_str \
            or "ニュース" in elem_str or "編集委員会" in elem_str or "入会申込" in elem_str \
            or "索引" in elem_str or "要旨"  in elem_str or "書評・紹介等" in elem_str:
                print(elem,"はスキップしました")
                continue
            else:
                output = pd.DataFrame(columns=features, index=[ind_num])

                url_art = elem.attrs['href'] #ジャーナルへのリンクを取得
                r_art = rq.get(url_art)
                time.sleep(1)
                soup_art = BeautifulSoup(r_art.text, "html.parser") #Beautifulsoupでの情報解析

                #発行日の取得
                day_tag = soup_art.find_all('meta', attrs={'name': 'publication_date'})
                day = day_tag[0].get('content')
                output["発行日"][ind_num] = day

                #巻と号の埋め込み
                output["巻"][ind_num] = kan
                output["号"][ind_num] = gou

                #タイトルの取得
                try:
                    title_tag = soup_art.find_all('meta', attrs={'name': 'title'})
                    title = title_tag[0].get('content')
                    output["タイトル"][ind_num] = title
                except:
                    pass

                #著者の取得
                authors_tag = soup_art.find_all('meta', attrs={'name': 'authors'})
                #著者は複数いるのでforで回す
                author_num = 1
                for author_tag in authors_tag:
                    author = author_tag.get('content')
                    #著者が4人以上だとスキップする
                    try:
                        output["著者{0}".format(author_num)][ind_num] = author
                        author_num = author_num + 1
                    except:
                        pass

                #doiの取得
                doi_tag = soup_art.find_all('meta', attrs={'name': 'citation_doi'})
                doi = doi_tag[0].get('content')
                output["doi"][ind_num] = doi

                #キーワードの取得
                keywords_tag = soup_art.find_all('meta', attrs={'name': 'keywords'})
                #キーワードは複数あるのでforで回す
                keyword_num = 1
                try: #キーワードがないのもあるので例外処理を行う
                    for keyword_tag in keywords_tag:
                        keyword = keyword_tag.get('content')
                        output["キーワード{0}".format(keyword_num)][ind_num] = keyword
                        keyword_num = keyword_num + 1
                except:
                    pass


                #抄録の取得
                abs_tag = soup_art.find_all('p', attrs={'class': 'global-para-14'})
                try: #抄録がないものもあるので例外処理を行う
                    abst = abs_tag[0].text
                    abst = abst.replace('\n', '') #改行コードが残っているので消す
                    output["抄録"][ind_num] = abst
                except:
                    pass

                #引用文献の取得
                references_tag = soup_art.find_all('meta', attrs={'name': 'references'})
                #引用文献は複数あるのでforで回す
                reference_num = 1
                try: #引用文献がないのもあるので例外処理を行う
                    for reference_tag in references_tag:
                        reference = reference_tag.get('content')
                        output["引用文献{0}".format(reference_num)][ind_num] = reference
                        reference_num = reference_num + 1
                except:
                    pass

                #過去のアウトプットとつなげる
                output_all = pd.concat([output_all, output], axis = 0)

                ind_num = ind_num + 1 #インデックス用の数値を1増やす

# 出力ファイル名指定、用変更
output_name = r"D:\Scraping\arp_vol{0}-{1}.csv".format(kan_start, kan_end)
output_all.to_csv(output_name, encoding ="cp932", errors = "ignore")

vol 39 issue 0
vol 39 issue 1
vol 39 issue 2
vol 39 issue 3
vol 39 issue 4
vol 40 issue 0
vol 40 issue 1
vol 40 issue 2
vol 40 issue 3
vol 40 issue 4
